# Sensors

In [6]:
import pandas as pd
import glob
import math
from datetime import datetime

# List all CSV files in the directory
csv_files = glob.glob('Sensor_Data/*.csv')

# Initialize an empty list to store DataFrames
dfs = []
# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    df = pd.read_csv(file,sep=";")
    dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

excel_data = pd.read_excel('Sensor_Data/Sensors.xlsx')

merged_data = pd.merge(combined_df, excel_data, left_on='mq_name', right_on='MQ_KURZNAME')




In [7]:
new_df = excel_data[['MQ_KURZNAME', 'LÄNGE (WGS84)', 'BREITE (WGS84)']]
new_df.to_csv('SUbi.csv', index=False)


In [8]:
def day_of_the_week(fecha):
    fecha_obj = datetime.strptime(fecha, '%d.%m.%Y')
    dia_semana = fecha_obj.strftime('%A')
    return day_to_week_num(dia_semana)

def day_to_week_num(day):
    weekdays = {
        'monday': 1,
        'tuesday': 2,
        'wednesday': 3,
        'thursday': 4,
        'friday': 5,
        'saturday': 6,
        'sunday': 7
    }
    return weekdays.get(day.lower(), None)

In [9]:
def month(fecha):
    # Convert the date string to a datetime object
    fecha_obj = datetime.strptime(fecha, '%d.%m.%Y')
    
    # Extract the month from the datetime object
    month = fecha_obj.month
    
    return month    

In [10]:
# Select only the columns you want to keep
columns_to_keep = ['mq_name', 'tag','stunde', 'q_kfz_mq_hr', 'LÄNGE (WGS84)', 'BREITE (WGS84)']

# Replace 'mq_name' with the actual column name you want to keep from excel_data
merged_data = merged_data[columns_to_keep]

# Rename for better Understanding
# Id, day,hour, num of motor vehicles, Longitud, Latitud
column_rename_mapping = {
    'mq_name': 'SensorID',
    'tag': 'SensorDate',
    'stunde':'SensorHour',
    'q_kfz_mq_hr': 'SensorNumOfMotorVehicles',
    'LÄNGE (WGS84)': 'SensorLongitude',
    'BREITE (WGS84)': 'SensorLatitude'
}

# Columns: Id, day, num_of_motor_vehicles
joined_data = merged_data.rename(columns=column_rename_mapping)

# We need to delate duplicates based on Id, day, hour
sensor_data = joined_data.drop_duplicates(subset=['SensorID', 'SensorDate','SensorHour'])
sensor_data.loc[:, 'SensorMonth'] = sensor_data['SensorDate'].apply(month)
sensor_data.loc[:, 'SensorDayOfWeek'] = sensor_data['SensorDate'].apply(lambda x: day_of_the_week(x))


C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17376\4089496898.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensor_data.loc[:, 'SensorMonth'] = sensor_data['SensorDate'].apply(month)
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17376\4089496898.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sensor_data.loc[:, 'SensorDayOfWeek'] = sensor_data['SensorDate'].apply(lambda x: day_of_the_week(x))


In [11]:
sensor_data.columns

Index(['SensorID', 'SensorDate', 'SensorHour', 'SensorNumOfMotorVehicles',
       'SensorLongitude', 'SensorLatitude', 'SensorMonth', 'SensorDayOfWeek'],
      dtype='object')

In [12]:
sensor_unique_coordinates = sensor_data[['SensorLongitude', 'SensorLatitude']].drop_duplicates()
sensor_unique_coordinates = list(sensor_unique_coordinates.itertuples(index=False, name=None))

# Accidents

In [13]:
# load necessary libraries
import pandas as pd
#import geopandas as gpd

# if you copied the folder 'Hertie-Data4Good-Festival-Challenge-1' to your main directory these paths should work
accidents_data = pd.read_csv('Accidents_Data/Accidents_Dataset.csv',sep=";")


In [14]:
# Cambia el nombre de las columnas para que estén en inglés
translated_columns = {
    "OBJECTID": "AccidentID",
    "LAND": "State",
    "BEZ": "District",
    "LOR_ab_2021": "LOR",
    "UJAHR": "AccidentYear",
    "UMONAT": "AccidentMonth",
    "USTUNDE": "AccidentHour",
    "UWOCHENTAG": "AccidentDayOfWeek",
    "UKATEGORIE": "AccidentCategory",
    "UART": "AccidentType",
    "UTYP1": "AccidentTypeDetail",
    "ULICHTVERH": "LightingCondition",
    "IstRad": "InvolvingBike",
    "IstPKW": "InvolvingCar",
    "IstFuss": "InvolvingPedestrian",
    "IstKrad": "InvolvingMotorcycle",
    "IstGkfz": "InvolvingHGV",
    "IstSonstige": "InvolvingOther",
    "USTRZUSTAND": "RoadCondition",
    "LINREFX": "GraphicCoord1",
    "LINREFY": "GraphicCoord2",
    "XGCSWGS84": "AccidentLongitude",
    "YGCSWGS84": "AccidentLatitude"
}

# Cambia los nombres de las columnas + Elimina null (Solo 1)
accidents_data.rename(columns=translated_columns, inplace=True)
accidents_data = accidents_data.dropna()


In [15]:
# Select only the columns you want to keep
columns_to_keep = ['AccidentID','AccidentCategory','AccidentLongitude','AccidentLatitude','AccidentDayOfWeek','AccidentHour', 'AccidentMonth']
accidents_data = accidents_data[columns_to_keep]

accidents_data['AccidentLongitude'] = accidents_data['AccidentLongitude'].str.replace(',', '.').astype(float)
accidents_data['AccidentLatitude'] = accidents_data['AccidentLatitude'].str.replace(',', '.').astype(float)


# Time to Join

In [16]:
import math
def inside_radius(coordenada_central, coordenada_punto, radio):
    """
    Verifica si una coordenada dada está dentro de un radio respecto a una coordenada central.
    
    Args:
    coordenada_central (tuple): Coordenadas (latitud, longitud) de la ubicación central en grados.
    coordenada_punto (tuple): Coordenadas (latitud, longitud) del punto a verificar en grados.
    radio (float): Radio en kilómetros en el que se debe encontrar el punto respecto a la coordenada central.
    
    Returns:
    bool: True si el punto está dentro del radio, False de lo contrario.
    """
    # Radio medio de la Tierra en kilómetros
    R = 6371.0
    
    # Convertir grados a radianes
    lat1_rad = math.radians(coordenada_central[0])
    lon1_rad = math.radians(coordenada_central[1])
    lat2_rad = math.radians(coordenada_punto[0])
    lon2_rad = math.radians(coordenada_punto[1])
    
    # Diferencias de latitud y longitud
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Calcular la distancia haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distancia = R * c
    
    # Verificar si la distancia es menor o igual al radio
    return [distancia <= radio, distancia]

In [17]:
# Iterate through each row in the DataFrame and modify the original dataset
radius = 0.2 #In km
for index, row in accidents_data.iterrows():
    # Extract latitude and longitude values from the current row
    longitude = row['AccidentLongitude']
    latitude = row['AccidentLatitude']
    min_ = 9999.9
    for coordinates in sensor_unique_coordinates:
        if inside_radius(coordinates, [longitude, latitude], radius)[0] and inside_radius(coordinates, [longitude, latitude], radius)[1]<min_:
            accidents_data.at[index, 'AccidentLongitude'] = coordinates[0]
            accidents_data.at[index, 'AccidentLatitude'] = coordinates[1]
            min_ = inside_radius(coordinates, [longitude, latitude],radius)[1]

In [18]:
accidents_data.dtypes

AccidentID             int64
AccidentCategory       int64
AccidentLongitude    float64
AccidentLatitude     float64
AccidentDayOfWeek      int64
AccidentHour           int64
AccidentMonth          int64
dtype: object

In [19]:
sensor_data.dtypes

SensorID                     object
SensorDate                   object
SensorHour                    int64
SensorNumOfMotorVehicles      int64
SensorLongitude             float64
SensorLatitude              float64
SensorMonth                   int64
SensorDayOfWeek               int64
dtype: object

In [20]:
SensorAccidents = pd.merge(sensor_data, accidents_data, left_on=['SensorDayOfWeek','SensorHour','SensorMonth','SensorLongitude', 'SensorLatitude'], right_on=['AccidentDayOfWeek','AccidentHour','AccidentMonth','AccidentLongitude','AccidentLatitude'])

# Select only the columns you want to keep
columns_to_keep = ['SensorID','SensorDate','AccidentDayOfWeek','AccidentHour','AccidentMonth','AccidentLatitude','AccidentLongitude','AccidentCategory','SensorNumOfMotorVehicles']
# Replace 'mq_name' with the actual column name you want to keep from excel_data
SensorAccidents = SensorAccidents[columns_to_keep]
# Rename for clarity
renamed_columns = {
    'SensorDate':'Date',
    "AccidentDayOfWeek": "DayOfWeek",
    "AccidentHour": "Hour",
    "AccidentMonth": "Month",
    "AccidentLatitude": "Latitude",
    "AccidentLongitude": "Longitude",
    "AccidentCategory": "AccidentCategory",
    "SensorNumOfMotorVehicles": "NumVehicles",
}

# Cambia los nombres de las columnas + Elimina null (Solo 1)
SensorAccidents.rename(columns=renamed_columns, inplace=True)


In [21]:
SensorAccidents.head()

,SensorID,Date,DayOfWeek,Hour,Month,Latitude,Longitude,AccidentCategory,NumVehicles
0,TE002,05.02.2021,5,17,2,52.433813,13.192747,3,1725
1,TE002,12.02.2021,5,17,2,52.433813,13.192747,3,1703
2,TE002,19.02.2021,5,17,2,52.433813,13.192747,3,1949
3,TE002,26.02.2021,5,17,2,52.433813,13.192747,3,1981
4,TE005,01.06.2021,2,18,6,52.435112,13.263106,2,975


In [22]:
print(SensorAccidents.shape)


(3068, 9)


In [23]:
sensor_unique_id = SensorAccidents[['SensorID']].drop_duplicates()
sensor_unique_id = list(sensor_unique_id.itertuples(index=False, name=None))
len(sensor_unique_id)

191

# Weather enters the chat

In [24]:
weather_data = pd.read_csv('Weather_Data/weather_data.csv',sep=",")
weather_data['rainy'] = weather_data['rainy'].astype(int)
renamed_columns = {
    'date':'Date',
    "hour": "Hour",
    "sunny": "Sunny",
    "rainy": "Rainy",
    "cloudy": "Cloudy"
}
weather_data.rename(columns=renamed_columns, inplace=True)


In [25]:
weather_data.head(5)

,Date,Hour,Sunny,Rainy,Cloudy
0,01.01.2021,0,0,0,1
1,01.01.2021,1,0,0,1
2,01.01.2021,2,0,0,1
3,01.01.2021,3,0,0,1
4,01.01.2021,4,0,0,1


In [26]:
SensorAccidents.head(5)

,SensorID,Date,DayOfWeek,Hour,Month,Latitude,Longitude,AccidentCategory,NumVehicles
0,TE002,05.02.2021,5,17,2,52.433813,13.192747,3,1725
1,TE002,12.02.2021,5,17,2,52.433813,13.192747,3,1703
2,TE002,19.02.2021,5,17,2,52.433813,13.192747,3,1949
3,TE002,26.02.2021,5,17,2,52.433813,13.192747,3,1981
4,TE005,01.06.2021,2,18,6,52.435112,13.263106,2,975


# Last Join, here we go!

In [27]:
SensorAccidentsWeather = pd.merge(SensorAccidents, weather_data, left_on=['Date','Hour'], right_on=['Date','Hour'])


In [28]:
SensorAccidentsWeather.head(5)

,SensorID,Date,DayOfWeek,Hour,Month,Latitude,Longitude,AccidentCategory,NumVehicles,Sunny,Rainy,Cloudy
0,TE002,05.02.2021,5,17,2,52.433813,13.192747,3,1725,0,1,0
1,TE002,12.02.2021,5,17,2,52.433813,13.192747,3,1703,0,0,1
2,TE002,19.02.2021,5,17,2,52.433813,13.192747,3,1949,0,0,1
3,TE002,26.02.2021,5,17,2,52.433813,13.192747,3,1981,0,0,1
4,TE005,01.06.2021,2,18,6,52.435112,13.263106,2,975,0,0,1


In [29]:
SensorAccidentsWeather.shape

(3063, 12)

In [30]:
# List of columns to drop
columns_to_drop = ['Date', 'Latitude', 'Longitude']

# Drop the specified columns
SensorAccidentsWeather.drop(columns=columns_to_drop, inplace=True)


In [31]:
SensorAccidentsWeather.head(5)

,SensorID,DayOfWeek,Hour,Month,AccidentCategory,NumVehicles,Sunny,Rainy,Cloudy
0,TE002,5,17,2,3,1725,0,1,0
1,TE002,5,17,2,3,1703,0,0,1
2,TE002,5,17,2,3,1949,0,0,1
3,TE002,5,17,2,3,1981,0,0,1
4,TE005,2,18,6,2,975,0,0,1


In [32]:
SensorAccidentsWeather.shape

(3063, 9)

In [33]:
# Assuming 'test' is your original DataFrame

# Create an empty list to store DataFrames of replicated rows
dfs_to_concatenate = []

# Replicate rows based on NumVehicles
for index, row in SensorAccidentsWeather.iterrows():
    num_vehicles = row['NumVehicles']
    for i in range(num_vehicles):
        replicated_row = row.copy()
        if i == 0 or i ==1:
            replicated_row['Label'] = 1
        else:
            replicated_row['Label'] = 0

        # Convert the replicated row to a DataFrame and append it to the list
        dfs_to_concatenate.append(pd.DataFrame(replicated_row).T)

# Concatenate the DataFrames in the list into a single DataFrame
SensorAccidentsWeather_expand = pd.concat(dfs_to_concatenate, ignore_index=True)


In [34]:
SensorAccidentsWeather_expand.drop(columns=['NumVehicles'], inplace=True)


In [35]:
SensorAccidentsWeather_expand.head()

,SensorID,DayOfWeek,Hour,Month,AccidentCategory,Sunny,Rainy,Cloudy,Label
0,TE002,5,17,2,3,0,1,0,1
1,TE002,5,17,2,3,0,1,0,1
2,TE002,5,17,2,3,0,1,0,0
3,TE002,5,17,2,3,0,1,0,0
4,TE002,5,17,2,3,0,1,0,0


In [36]:
# Define the file path where you want to store the dataset
file_path = 'SensorAccidentsWeather_expand.csv'
# Store the dataset to a CSV file
SensorAccidentsWeather_expand.to_csv(file_path, index=False)

In [37]:
data = pd.read_csv('SensorAccidentsWeather_expand.csv',sep=",")
data.head()

,SensorID,DayOfWeek,Hour,Month,AccidentCategory,Sunny,Rainy,Cloudy,Label
0,TE002,5,17,2,3,0,1,0,1
1,TE002,5,17,2,3,0,1,0,1
2,TE002,5,17,2,3,0,1,0,0
3,TE002,5,17,2,3,0,1,0,0
4,TE002,5,17,2,3,0,1,0,0


In [38]:
# rainy, sunny, cloudy
def convert_weather_format(rainy, sunny, cloudy):
    weather_mapping = {
        (0, 0, 1): [1,0,0],
        (1, 0, 0): [0,1,0],
        (0, 1, 0): [0,0,1]
    }
    weather = weather_mapping[(rainy, sunny, cloudy)]
   
    return weather


In [39]:
# Assuming SensorAccidentsWeather_expand is your DataFrame
data['Weather'] = data.apply(
    lambda row: convert_weather_format(row['Rainy'], row['Sunny'], row['Cloudy']),
    axis=1)
#data['AccidentCategory'] = data['AccidentCategory'].apply(convert_category)

# Drop the 'Sunny', 'Rainy', and 'Cloudy' columns if needed
data.drop(['Sunny', 'Rainy', 'Cloudy','AccidentCategory'], axis=1, inplace=True)

In [40]:
data.head(5)

,SensorID,DayOfWeek,Hour,Month,Label,Weather
0,TE002,5,17,2,1,"[0, 1, 0]"
1,TE002,5,17,2,1,"[0, 1, 0]"
2,TE002,5,17,2,0,"[0, 1, 0]"
3,TE002,5,17,2,0,"[0, 1, 0]"
4,TE002,5,17,2,0,"[0, 1, 0]"


# Save the final csv

In [41]:
# Create a mapping dictionary for SensorID to numeric identifier
sensor_id_map = {sensor_id: idx + 1 for idx, sensor_id in enumerate(data['SensorID'].unique())}

# Replace SensorID values with numeric identifiers
data['SensorID'] = data['SensorID'].map(sensor_id_map)

In [42]:
data.head()
def convert_weather(vec):
    weather_mapping = {
        (1, 0, 0): 1,  # rainy
        (0, 1, 0): 2,  # sunny
        (0, 0, 1): 3   # cloudy
    }
    weather = weather_mapping[tuple(vec)]
    return weather

In [43]:
data.head(5)

,SensorID,DayOfWeek,Hour,Month,Label,Weather
0,1,5,17,2,1,"[0, 1, 0]"
1,1,5,17,2,1,"[0, 1, 0]"
2,1,5,17,2,0,"[0, 1, 0]"
3,1,5,17,2,0,"[0, 1, 0]"
4,1,5,17,2,0,"[0, 1, 0]"


In [ ]:
data.dtypes

In [45]:
# Apply the convert_weather function to the 'Weather' column
data['Weather'] = data['Weather'].apply(convert_weather)


In [48]:
data.to_csv('InputData.csv', index=False)
